In [6]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('order_items.csv', 'order_items'),
    ('payments.csv', 'payments')  # Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Rashmi1!',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/admin/Desktop/Rashmi_Ecommerce'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.1 MB 3.3 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/16.1 MB 3.7 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/16.1 MB 3.8 MB/s eta 0:00:04
   ----- ---------------------------------- 2.4/16.1 MB 3.0 MB/s eta 0:00:05
   ------- -------------------------------- 2.9/16.1 MB 3.1 MB/s eta 0:00:05
   -------- ------------------------------- 3.4/16.1 MB 2.9 MB/s eta 0:00:05
   --------- ------------------------------ 3.9/16.1 MB 2.7 MB/s eta 0:00:05
   ----------- ---------------------------- 4.5/16.1 MB 2.7 MB/s eta 0:00:05
   ------------ --------------------------- 5.0/16.1 MB 2.7 MB/s eta 0:00:05
   ------------- -------------------------- 5.5/16.1 MB 2.7 MB/s eta 0:00:04
   --------------- ------------------------ 6.3/16.1 MB 2.8 MB/s eta 0:00:04
   ----------------- ---------------------- 7.1/16.1 MB 2.9 MB/s eta 0:00:04
   ---

In [8]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import mysql.connector


In [10]:
db=mysql.connector.connect(host="localhost",username="root",password = "Rashmi1!", database="ecommerce")
cur=db.cursor()

In [12]:
#List all unique cities where customers are located.
query="""select distinct (customer_city) from customers"""
cur.execute(query)
data=cur.fetchall()
data

[('franca',),
 ('sao bernardo do campo',),
 ('sao paulo',),
 ('mogi das cruzes',),
 ('campinas',),
 ('jaragua do sul',),
 ('timoteo',),
 ('curitiba',),
 ('belo horizonte',),
 ('montes claros',),
 ('rio de janeiro',),
 ('lencois paulista',),
 ('caxias do sul',),
 ('piracicaba',),
 ('guarulhos',),
 ('pacaja',),
 ('florianopolis',),
 ('aparecida de goiania',),
 ('santo andre',),
 ('goiania',),
 ('cachoeiro de itapemirim',),
 ('sao jose dos campos',),
 ('sao roque',),
 ('camacari',),
 ('resende',),
 ('sumare',),
 ('novo hamburgo',),
 ('sao luis',),
 ('sao jose',),
 ('santa barbara',),
 ('ribeirao preto',),
 ('ituiutaba',),
 ('taquarituba',),
 ('sao jose dos pinhais',),
 ('barrinha',),
 ('parati',),
 ('dourados',),
 ('trindade',),
 ('cascavel',),
 ('fortaleza',),
 ('brasilia',),
 ('pelotas',),
 ('porto alegre',),
 ('salto',),
 ('jundiai',),
 ('cacapava',),
 ('sao vicente',),
 ('uberlandia',),
 ('botelhos',),
 ('sao goncalo',),
 ('araucaria',),
 ('nova iguacu',),
 ('areia branca',),
 ('campo

In [16]:
# Count the number of orders placed in 2017.
query="""select count(order_id) from orders where year(order_purchase_timestamp)=2017"""
cur.execute(query)
data=cur.fetchall()
data

[(45101,)]

In [18]:
data[0]

(45101,)

In [20]:
data[0][0]

45101

In [24]:
query="""select products.product_category category, 
round(sum(payments.payment_value),2) sales
from products join order_items
on products.product_id=order_items.product_id
join  payments
on payments.order_id=order_items.order_id
group by category"""
cur.execute(query)
data=cur.fetchall()
data

[('perfumery', 506738.66),
 ('Furniture Decoration', 1430176.39),
 ('telephony', 486882.05),
 ('bed table bath', 1712553.67),
 ('automotive', 852294.33),
 ('computer accessories', 1585330.45),
 ('housewares', 1094758.13),
 ('babies', 539845.66),
 ('toys', 619037.69),
 ('Furniture office', 646826.49),
 ('Cool Stuff', 779698.0),
 ('HEALTH BEAUTY', 1657373.12),
 ('pet Shop', 311268.97),
 ('General Interest Books', 64694.76),
 ('stationary store', 317440.07),
 ('Watches present', 1429216.68),
 ('PCs', 279121.55),
 ('climatization', 91170.66),
 ('Construction Tools Construction', 241475.63),
 ('sport leisure', 1392127.56),
 ('Bags Accessories', 187151.29),
 ('electronics', 259857.1),
 ('Casa Construcao', 136645.29),
 ('Games consoles', 195480.38),
 ('Fashion Bags and Accessories', 218158.28),
 ('home appliances', 95532.27),
 ('Garden tools', 838280.75),
 ('Agro Industria e Comercio', 118730.61),
 (None, 252801.71),
 ('audio', 60324.62),
 ('technical books', 24915.44),
 ('Construction Tools 

In [46]:
df=pd.DataFrame(data,columns=["Category","Sales"])
df

,Category,Sales
0,perfumery,506738.66
1,Furniture Decoration,1430176.39
2,telephony,486882.05
3,bed table bath,1712553.67
4,automotive,852294.33
...,...,...
69,cds music dvds,1199.43
70,La Cuisine,2913.53
71,Fashion Children's Clothing,785.67
72,PC Gamer,2174.43


In [50]:
query=""" select orders.order_id,orders.customer_id, count(order_items.order_id) from orders join order_items on orders.order_id=order_items.order_id group by orders.order_id,orders.customer_id"""
cur.execute(query)
data=cur.fetchall()
data

[('00125cb692d04887809806618a2a145f', '8afb90a97ee661103014329b1bcea1a2', 1),
 ('001c85b5f68d2be0cb0797afc9e8ce9a', '48ed31e735f1c420ed6ca3637b7c744d', 1),
 ('00471463a6106056c1a2a809f70de640', 'fcad74a7e0dff27c33e36cbd78280bf9', 1),
 ('00571ded73b3c061925584feab0db425', '3c628393675b42c6b5ef89461f68ecef', 2),
 ('006dd93155bc2abd844cc5eed3a0fe7f', '1b9506de3da8fd318853195b13fb2dda', 1),
 ('008fab5602c441d51f30b1464f5d4643', '337e39b83fac394e62eb7876f45ee981', 1),
 ('00946f674d880be1f188abc10ad7cf46', '9583b047c90d4aa1c5c68b4065ba7f4e', 2),
 ('00a0116ff15ff973ea16bee881208ae7', '89a187407adfc8a32a4d6233a6a237d8', 1),
 ('00bdcdda88e6b02977fc6ce3d412c600', '45ba03e2c6bbb5dc48131ba32ec3ae5e', 1),
 ('011b142c9e082a5c1d10e0a88cd9c8e8', '0c7a95280b3f80eadd81c829e093dbec', 1),
 ('0132451f29a10b66a5cf1bacc85f9afe', '1c5b37d20011c637ad1a5b6d423c7483', 2),
 ('01835ef41c5ac66dd9ec6084fa1d0533', '6a4ea2081853cd8dd8d7c69cdf86925a', 1),
 ('01855f880aae9a984c7c33b26fcf2e02', '2e068340b3037f667ce3224bd

In [ ]:
with count_per_order as (select orders.order_id,orders.customer_id, count(order_items.order_id) from orders join order_items on orders.order_id=order_items.order_id group by orders.order_id,orders.customer_id)